# Setup for s2i notebook deployment on OpenShift

This notebook outlines the necessary steps and command line arguments that a user would need to deploy their machine learning notebooks as a flask app on OpenShift. It will cover the following:

* Access to OpenShift
* Install oc client
* Git clone a repo
* Set env Variables
* Deploy new app
* Expose it as a service
* Interact with the service

## Access to OpenShift 

If you don't already have access to OpenShift you can go to https://www.okd.io/ and try it out yourself or sign up for a free account with OpenShift online (https://www.openshift.com/products/online/)

Once you have access to OpenShift you are going to want to use the oc client on your local machine or in a jupyter hub terminal to interact with it.  

## Install oc client

In [ ]:
%%bash 
curl -o oc.tar.gz -L https://mirror.openshift.com/pub/openshift-v4/clients/ocp/latest/openshift-client-linux-4.1.0-rc.0.tar.gz 
tar xzf oc.tar.gz 
cp oc ~/../bin/oc

## oc commands to login, check your connection to OpenShift and select your project

In [ ]:
! oc login hostname -u user-name -p password

In [ ]:
! oc whoami

In [ ]:
! oc project your-project-name

## Clone the correct repo into your local environment or jupyter hub

In this example we will be using one of my repos: https://github.com/MichaelClifford/example-model-s2i-notebook

However, it is important to note that any repo that follows the same simple conventions can be used.

In [ ]:
! git clone https://github.com/MichaelClifford/example-model-s2i-notebook

Once the user has made the appropriate changes to their notebook they can push it back to github for the s2i builder. 

## Set Environment Variables as Secrets

Any environment variables needed for the notebook to execute such as s3 bucket credentials can be configured as [Secrets](https://docs.openshift.com/container-platform/3.11/dev_guide/secrets.html) in OpenShift. Using the web console, by navigating to Resources and Secrets, we can create "Generic" Secret types for the environment variable, set a "Secret Name", specify the name of the key referenced in the notebook as "Key" and the value of the environment variable as "Value".

After launching the service, we can add these Secrets to the Build Environment Variables and selecting "Add Value from Config Map or Secret". We can let the initial build fail and upon triggering a "Rebuild", the secrets will be used.

## Build notebook image and launch service

On OpenShift, with this simple command below we can invoke a special image builder for notebooks and provided it a github repository containing a machine learning notebook with data collection, processing, training and inference and build an image and deploy it directly as an inference service running a flask application in a pod.    

In [ ]:
%%bash
oc new-app --name model \
quay.io/willbenton/simple-model-s2i:demo\
~https://github.com/MichaelClifford/example-model-s2i-notebook

When building from a private Git repo, we need to follow some additional steps. We can use deploy keys to authenticate the private Git repository.
By following the steps outlined [here](https://www.openshift.com/blog/private-git-repositories-part-2a-repository-ssh-keys), we can generate an ssh key to be used with the repository. We need to add the Public key (the .pub file) that we generated to the Github Repo by going to the repo settings under "Add deploy key".  

The private key can be added as a Source Secret on OpenShift the same way we added environment variables. For adding the private ssh key, we can create a new Source Secret and select authentication type as "SSH Key", specify a secret name and paste the private key that we generated in the Private SSH Key section.

Once we have set up the Secret, we can run the `new-app` command with the ssh url of the repo and specify the --source-secret parameter.

In [ ]:
%%bash
oc new-app --name model \
quay.io/willbenton/simple-model-s2i:demo\
~git@github.com:MichaelClifford/example-model-s2i-notebook.git \
--source-secret secretname

## Expose service for interaction with inference engine

Finally, just so it can be used by others we need to expose the route of our service so that it can be accessed by other users with the following command. 

In [ ]:
! oc expose svc/model 

## Interact with model

This basic code snippet can then be used to interact with the flask app, or if you prefer you could use your favorite REST tool like Insomnia (https://insomnia.rest/)

In [ ]:
import base64
import requests
import cloudpickle
import numpy as np

In [ ]:
baseurl = "your-models-route"
args = np.ones(29)
payload = {'args': base64.b64encode(cloudpickle.dumps(args))}
r = requests.post(baseurl + "/predict", data=payload)
print(" input is class", r.text)